In [2]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']

word_dict = {n:i for i,n in enumerate(char_arr)}

number_dict = {i:n for i,n in enumerate(char_arr)}

n_class = len(word_dict)

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

#parameter
n_step =3
n_hidden = 128

In [9]:
def make_batch(seq_data):
    input_batch, target_batch = [], []
    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]]
        target = word_dict[seq[-1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])
    return input_batch,target_batch

In [10]:
#model
X = tf.placeholder(tf.float32, shape=[None, n_step, n_class])
Y = tf.placeholder(tf.float32, shape=[None, n_class])

W = tf.Variable(tf.random_uniform([n_hidden, n_class]))
b = tf.Variable(tf.random_uniform([n_class]))

cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) #  lstm单元构造
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

#outputs : [batch_size, n_step, n_hidden]
outputs = tf.transpose(outputs, [1,0,2]) #[n_step, batch_size, n_hidden]
outputs = outputs[-1] #[batch_size, n_hidden]
model = tf.matmul(outputs, W)+b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.cast(tf.argmax(model,1), tf.int32)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(seq_data)

In [21]:
for epoch in range(1000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch+1) % 200 == 0:
        print('Epoch : {:5d}, cost={:.5f}'.format(epoch+1, loss))

Epoch :   200, cost=0.00016
Epoch :   400, cost=0.00012
Epoch :   600, cost=0.00009
Epoch :   800, cost=0.00007
Epoch :  1000, cost=0.00006


In [25]:
inputs = [sen[:3] for sen in seq_data]

predict = sess.run([prediction], feed_dict={X: input_batch})

print(inputs, '->', [number_dict[n] for n in predict[0]])

['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
